# Train Pretrained Vertical Model for oral bioavailability prediction

1. This notebook used the verticalGNN model pretrained with mid similarity solubility dataset containing 9844 molecules

In [1]:
# import all required materials

import torch
import numpy as np
from torch_geometric.loader import DataLoader
from sklearn.model_selection import KFold
import os

from model import VerticalGNN
from config import NUM_FEATURES, NUM_TARGET, EDGE_DIM, DEVICE, SEED_NO, PATIENCE, EPOCHS, NUM_GRAPHS_PER_BATCH, N_SPLITS, best_params_vertical
from engine import EngineSol, EngineHOB
from utils import seed_everything, LoadHOBDataset, LoadSolDataset

First, we define a train and test function that can aid us in transfer learning. Then, we evaluate how different factors can affect the results of transfer learning. 

In [9]:
def run_training(method_tf, train_loader, valid_loader, params,es_trigger, path_to_pretrained_model, path_to_save_trained_model):
    
    '''
    Define a function to wrap training

    Args:
    method_tf (str): freeze --> freeze parameters of feature extraction block, fine_tune_5x -> fine tune at 5x slower learning rate, 
                    fine_tune_10x -> fine tune at 10x slower learning rate
    train_loader: DataLoader class from pytorch geometric containing train data
    valid_loader: DataLoader class from pytorch geometric containing validation data
    params (dict): dictionary containing the hyperparameters
    es_trigger (int): a number to force train model before triggering early stopping mechanism 
    path_to_pretrained_model (str): path to load the pretrained models
    path_to_save_trained_model: path to save the trained models

    Return:
    best loss: return best validation loss
    '''
    
    model = VerticalGNN(
            num_features=NUM_FEATURES,
            num_targets=NUM_TARGET,
            num_gin_layers=params["num_gin_layers"],
            num_graph_trans_layers=params["num_graph_trans_layers"],
            hidden_size=params["hidden_size"],
            n_heads=params["n_heads"],
            dropout=params["dropout"],
            edge_dim=EDGE_DIM,
        )

    model.load_state_dict(torch.load(path_to_pretrained_model))  
    model.to(DEVICE)
    if method_tf == 'freeze':
        for param in model.gin_model.parameters():
            param.requires_grad=False
        for param in model.graph_trans_model.parameters():
            param.requires_grad=False

        optimizer=torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr = params['learning_rate'])
    
    elif method_tf == 'fine_tune_5x':
        optimizer=torch.optim.Adam([
            {'params': model.gin_model.parameters(), 'lr': params['learning_rate']/5},
            {'params': model.graph_trans_model.parameters(), 'lr': params['learning_rate']/5},
            {'params': model.ro.parameters()}
        ],lr = params['learning_rate'])

    elif method_tf == 'fine_tune_10x':
        optimizer=torch.optim.Adam([
            {'params': model.gin_model.parameters(), 'lr': params['learning_rate']/10},
            {'params': model.graph_trans_model.parameters(), 'lr': params['learning_rate']/10},
            {'params': model.ro.parameters()}
        ],lr = params['learning_rate'])

    elif method_tf == 'fine_tune':
        optimizer=torch.optim.Adam([
            {'params': model.gin_model.parameters(), 'lr': params['learning_rate']},
            {'params': model.graph_trans_model.parameters(), 'lr': params['learning_rate']},
            {'params': model.ro.parameters()}
        ],lr = params['learning_rate'])
        
    eng = EngineHOB(model, optimizer, device=DEVICE)

    best_loss = np.inf
    early_stopping_iter = PATIENCE
    early_stopping_counter = 0

    for epoch in range(EPOCHS):
        train_loss = eng.train(train_loader)
        valid_loss = eng.validate(valid_loader)
        print(
            f"Epoch: {epoch+1}/{EPOCHS}, train loss : {train_loss}, validation loss : {valid_loss}"
        )
        if epoch+1>es_trigger:
            if valid_loss < best_loss:
                best_loss = valid_loss
                early_stopping_counter = 0  # reset counter
                print("Saving model...")
                torch.save(model.state_dict(), path_to_save_trained_model)
            else:
                early_stopping_counter += 1

            if early_stopping_counter > early_stopping_iter:
                print("Early stopping...")
                break
            print(f"Early stop counter: {early_stopping_counter}")

    return best_loss



In [7]:
def run_testing(method_tf, test_loader, params, path_to_trained_model):
    
    '''
    Define a function to wrap training

    Args:
    method_tf (str): freeze --> freeze parameters of feature extraction block, fine_tune_5x -> fine tune at 5x slower learning rate, 
                    fine_tune_10x -> fine tune at 10x slower learning rate
    test_loader: DataLoader class from pytorch geometric containing test data
    params (dict): dictionary containing the hyperparameters
    path_to_save_trained_model: path to load the saved trained models

    Return:
    bce: logloss from pytorch geometric 
    acc: accuracy score
    f1: f1 score
    roc_auc: roc auc score
    '''
    
    model = VerticalGNN(
            num_features=NUM_FEATURES,
            num_targets=NUM_TARGET,
            num_gin_layers=params["num_gin_layers"],
            num_graph_trans_layers=params["num_graph_trans_layers"],
            hidden_size=params["hidden_size"],
            n_heads=params["n_heads"],
            dropout=params["dropout"],
            edge_dim=EDGE_DIM,
        )

    model.load_state_dict(torch.load(path_to_trained_model))  
    model.to(DEVICE)
    if method_tf == 'freeze':
        for param in model.gin_model.parameters():
            param.requires_grad=False
        for param in model.graph_trans_model.parameters():
            param.requires_grad=False

        optimizer=torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr = params['learning_rate'])        
    
    elif method_tf == 'fine_tune_5x':
        optimizer=torch.optim.Adam([
            {'params': model.gin_model.parameters(), 'lr': params['learning_rate']/5},
            {'params': model.graph_trans_model.parameters(), 'lr': params['learning_rate']/5},
            {'params': model.ro.parameters()}
        ],lr = params['learning_rate'])
        

    elif method_tf == 'fine_tune_10x':
        optimizer=torch.optim.Adam([
            {'params': model.gin_model.parameters(), 'lr': params['learning_rate']/10},
            {'params': model.graph_trans_model.parameters(), 'lr': params['learning_rate']/10},
            {'params': model.ro.parameters()}
        ],lr = params['learning_rate'])
    
    elif method_tf == 'fine_tune':
        optimizer=torch.optim.Adam([
            {'params': model.gin_model.parameters(), 'lr': params['learning_rate']},
            {'params': model.graph_trans_model.parameters(), 'lr': params['learning_rate']},
            {'params': model.ro.parameters()}
        ],lr = params['learning_rate'])
    
        

    eng = EngineHOB(model, optimizer, device=DEVICE)
    bce, acc, f1, roc_auc = eng.test(test_loader)
    print(f"bce:{bce}, acc :{acc}, f1: {f1}, roc_auc: {roc_auc}")
    return bce, acc, f1, roc_auc

# Effect of number of pre-training epochs to transfer learning prediction performance
1. Weights are first frozen for the feature extraction block. 
2. Models are allowed to train and only weights for the classifier block is allowed to be updated. 
3. To evaluate how number of pre-training epochs affect the transfer learning prediction performance.

In [4]:
train_data_root_path = './data/graph_data/data_oral_avail_train/'
train_data_raw_filename = 'data_oral_avail_train_50.csv'
test_data_root_path = './data/graph_data/data_oral_avail_test'
test_data_raw_filename = 'data_oral_avail_test_1_50.csv'
n_repetitions = 5
method_tf = 'freeze'
params = best_params_vertical
es_trigger = 0
path_to_pretrained_model = './trf_learning_models/pretrained_models/vertical/mid_10x/'
path_to_save_trained_model = './trf_learning_models/trained_models/vertical/mid_10x/pretrained_20/'
path_to_trained_model = './trf_learning_models/trained_models/vertical/mid_10x/pretrained_20/'


bce_list = []
acc_list = []
f1_list = []
roc_auc_list = []

dataset_for_cv = LoadHOBDataset(train_data_root_path, train_data_raw_filename)
kf = KFold(n_splits=N_SPLITS)

for repeat in range(n_repetitions):
    for fold_no, (train_idx, valid_idx) in enumerate(kf.split(dataset_for_cv)):
        seed_everything(SEED_NO)
        train_dataset = []
        valid_dataset = []
        
        for t_idx in train_idx:
            train_dataset.append(
                torch.load(
                    f"./data/graph_data/data_oral_avail_train/processed/molecule_{t_idx}.pt"
                )
            )
        for v_idx in valid_idx:
            valid_dataset.append(
                torch.load(
                    f"./data/graph_data/data_oral_avail_train/processed/molecule_{v_idx}.pt"
                )
            )

        train_loader = DataLoader(
            train_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=True
        )
        valid_loader = DataLoader(
            valid_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=False
        )
        test_dataset = LoadHOBDataset(test_data_root_path, test_data_raw_filename)
        test_loader = DataLoader(
            test_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=False
        )
        print(f'Rep no {repeat}, Fold no {fold_no}')
        run_training(method_tf, train_loader, valid_loader, params, es_trigger, os.path.join(path_to_pretrained_model, f'pretrained_vertical_model_20_epoch.pt'),
            os.path.join(
                path_to_save_trained_model, f"trained_vertical_model_{method_tf}_repeat_{repeat}_fold_{fold_no}_{es_trigger}_es_trigger.pt"
            ),
        )

        bce, acc, f1, roc_auc = run_testing(method_tf, test_loader, params, 
                        os.path.join(path_to_save_trained_model, f"trained_vertical_model_{method_tf}_repeat_{repeat}_fold_{fold_no}_{es_trigger}_es_trigger.pt"))

        bce_list.append(bce)
        acc_list.append(acc)
        f1_list.append(f1)
        roc_auc_list.append(roc_auc)

bce_arr = np.array(bce_list)
mean_bce = np.mean(bce_arr)
sd_bce = np.std(bce_arr)
print(f'bce:{mean_bce:.3f}±{sd_bce:.3f}')

acc_arr = np.array(acc_list)
acc_mean= np.mean(acc_arr)
acc_sd = np.std(acc_arr)
print(f'acc:{acc_mean:.3f}±{acc_sd:.3f}')

f1_arr = np.array(f1_list)
f1_mean= np.mean(f1_arr)
f1_sd = np.std(f1_arr)
print(f'f1: {f1_mean:.3f}±{f1_sd:.3f}')

roc_auc_arr = np.array(roc_auc_list)
roc_auc_mean= np.mean(roc_auc_arr)
roc_auc_sd = np.std(roc_auc_arr)
print(f'roc_auc: {roc_auc_mean:.3f}±{roc_auc_sd:.3f}')

print("Training Completed!")

Rep no 0, Fold no 0
Epoch: 1/300, train loss : 1.4394868612289429, validation loss : 0.7983830571174622
Saving model...
Early stop counter: 0
Epoch: 2/300, train loss : 0.7587292194366455, validation loss : 0.661930501461029
Saving model...
Early stop counter: 0
Epoch: 3/300, train loss : 0.820805087685585, validation loss : 0.6230358481407166
Saving model...
Early stop counter: 0
Epoch: 4/300, train loss : 0.6946954727172852, validation loss : 0.6573702096939087
Early stop counter: 1
Epoch: 5/300, train loss : 0.6959695518016815, validation loss : 0.6967557668685913
Early stop counter: 2
Epoch: 6/300, train loss : 0.6878052800893784, validation loss : 0.6331246495246887
Early stop counter: 3
Epoch: 7/300, train loss : 0.6676203459501266, validation loss : 0.601194441318512
Saving model...
Early stop counter: 0
Epoch: 8/300, train loss : 0.6635631173849106, validation loss : 0.602630615234375
Early stop counter: 1
Epoch: 9/300, train loss : 0.6547203361988068, validation loss : 0.62249

In [5]:
train_data_root_path = './data/graph_data/data_oral_avail_train/'
train_data_raw_filename = 'data_oral_avail_train_50.csv'
test_data_root_path = './data/graph_data/data_oral_avail_test'
test_data_raw_filename = 'data_oral_avail_test_1_50.csv'
n_repetitions = 5
method_tf = 'freeze'
params = best_params_vertical
es_trigger = 0
path_to_pretrained_model = './trf_learning_models/pretrained_models/vertical/mid_10x/'
path_to_save_trained_model = './trf_learning_models/trained_models/vertical/mid_10x/pretrained_40/'
path_to_trained_model = './trf_learning_models/trained_models/vertical/mid_10x/pretrained_40/'


bce_list = []
acc_list = []
f1_list = []
roc_auc_list = []

dataset_for_cv = LoadHOBDataset(train_data_root_path, train_data_raw_filename)
kf = KFold(n_splits=N_SPLITS)

for repeat in range(n_repetitions):
    for fold_no, (train_idx, valid_idx) in enumerate(kf.split(dataset_for_cv)):
        seed_everything(SEED_NO)
        train_dataset = []
        valid_dataset = []
        
        for t_idx in train_idx:
            train_dataset.append(
                torch.load(
                    f"./data/graph_data/data_oral_avail_train/processed/molecule_{t_idx}.pt"
                )
            )
        for v_idx in valid_idx:
            valid_dataset.append(
                torch.load(
                    f"./data/graph_data/data_oral_avail_train/processed/molecule_{v_idx}.pt"
                )
            )

        train_loader = DataLoader(
            train_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=True
        )
        valid_loader = DataLoader(
            valid_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=False
        )
        test_dataset = LoadHOBDataset(test_data_root_path, test_data_raw_filename)
        test_loader = DataLoader(
            test_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=False
        )
        print(f'Rep no {repeat}, Fold no {fold_no}')
        run_training(method_tf, train_loader, valid_loader, params, es_trigger, os.path.join(path_to_pretrained_model, f'pretrained_vertical_model_40_epoch.pt'),
            os.path.join(
                path_to_save_trained_model, f"trained_vertical_model_{method_tf}_repeat_{repeat}_fold_{fold_no}_{es_trigger}_es_trigger.pt"
            ),
        )

        bce, acc, f1, roc_auc = run_testing(method_tf, test_loader, params, 
                        os.path.join(path_to_save_trained_model, f"trained_vertical_model_{method_tf}_repeat_{repeat}_fold_{fold_no}_{es_trigger}_es_trigger.pt"))

        bce_list.append(bce)
        acc_list.append(acc)
        f1_list.append(f1)
        roc_auc_list.append(roc_auc)

bce_arr = np.array(bce_list)
mean_bce = np.mean(bce_arr)
sd_bce = np.std(bce_arr)
print(f'bce:{mean_bce:.3f}±{sd_bce:.3f}')

acc_arr = np.array(acc_list)
acc_mean= np.mean(acc_arr)
acc_sd = np.std(acc_arr)
print(f'acc:{acc_mean:.3f}±{acc_sd:.3f}')

f1_arr = np.array(f1_list)
f1_mean= np.mean(f1_arr)
f1_sd = np.std(f1_arr)
print(f'f1: {f1_mean:.3f}±{f1_sd:.3f}')

roc_auc_arr = np.array(roc_auc_list)
roc_auc_mean= np.mean(roc_auc_arr)
roc_auc_sd = np.std(roc_auc_arr)
print(f'roc_auc: {roc_auc_mean:.3f}±{roc_auc_sd:.3f}')

print("Training Completed!")

Rep no 0, Fold no 0
Epoch: 1/300, train loss : 1.508273422718048, validation loss : 0.8205875754356384
Saving model...
Early stop counter: 0
Epoch: 2/300, train loss : 0.7794691324234009, validation loss : 0.7330502271652222
Saving model...
Early stop counter: 0
Epoch: 3/300, train loss : 0.8416727483272552, validation loss : 0.696063220500946
Saving model...
Early stop counter: 0
Epoch: 4/300, train loss : 0.7162857800722122, validation loss : 0.6603297591209412
Saving model...
Early stop counter: 0
Epoch: 5/300, train loss : 0.6901327073574066, validation loss : 0.6821579337120056
Early stop counter: 1
Epoch: 6/300, train loss : 0.7000085562467575, validation loss : 0.6535473465919495
Saving model...
Early stop counter: 0
Epoch: 7/300, train loss : 0.6710651814937592, validation loss : 0.6256861090660095
Saving model...
Early stop counter: 0
Epoch: 8/300, train loss : 0.6533190011978149, validation loss : 0.6156219840049744
Saving model...
Early stop counter: 0
Epoch: 9/300, train lo

In [6]:
train_data_root_path = './data/graph_data/data_oral_avail_train/'
train_data_raw_filename = 'data_oral_avail_train_50.csv'
test_data_root_path = './data/graph_data/data_oral_avail_test'
test_data_raw_filename = 'data_oral_avail_test_1_50.csv'
n_repetitions = 5
method_tf = 'freeze'
params = best_params_vertical
es_trigger = 0
path_to_pretrained_model = './trf_learning_models/pretrained_models/vertical/mid_10x/'
path_to_save_trained_model = './trf_learning_models/trained_models/vertical/mid_10x/pretrained_60/'
path_to_trained_model = './trf_learning_models/trained_models/vertical/mid_10x/pretrained_60/'


bce_list = []
acc_list = []
f1_list = []
roc_auc_list = []

dataset_for_cv = LoadHOBDataset(train_data_root_path, train_data_raw_filename)
kf = KFold(n_splits=N_SPLITS)

for repeat in range(n_repetitions):
    for fold_no, (train_idx, valid_idx) in enumerate(kf.split(dataset_for_cv)):
        seed_everything(SEED_NO)
        train_dataset = []
        valid_dataset = []
        
        for t_idx in train_idx:
            train_dataset.append(
                torch.load(
                    f"./data/graph_data/data_oral_avail_train/processed/molecule_{t_idx}.pt"
                )
            )
        for v_idx in valid_idx:
            valid_dataset.append(
                torch.load(
                    f"./data/graph_data/data_oral_avail_train/processed/molecule_{v_idx}.pt"
                )
            )

        train_loader = DataLoader(
            train_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=True
        )
        valid_loader = DataLoader(
            valid_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=False
        )
        test_dataset = LoadHOBDataset(test_data_root_path, test_data_raw_filename)
        test_loader = DataLoader(
            test_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=False
        )
        print(f'Rep no {repeat}, Fold no {fold_no}')
        run_training(method_tf, train_loader, valid_loader, params, es_trigger, os.path.join(path_to_pretrained_model, f'pretrained_vertical_model_60_epoch.pt'),
            os.path.join(
                path_to_save_trained_model, f"trained_vertical_model_{method_tf}_repeat_{repeat}_fold_{fold_no}_{es_trigger}_es_trigger.pt"
            ),
        )

        bce, acc, f1, roc_auc = run_testing(method_tf, test_loader, params, 
                        os.path.join(path_to_save_trained_model, f"trained_vertical_model_{method_tf}_repeat_{repeat}_fold_{fold_no}_{es_trigger}_es_trigger.pt"))

        bce_list.append(bce)
        acc_list.append(acc)
        f1_list.append(f1)
        roc_auc_list.append(roc_auc)

bce_arr = np.array(bce_list)
mean_bce = np.mean(bce_arr)
sd_bce = np.std(bce_arr)
print(f'bce:{mean_bce:.3f}±{sd_bce:.3f}')

acc_arr = np.array(acc_list)
acc_mean= np.mean(acc_arr)
acc_sd = np.std(acc_arr)
print(f'acc:{acc_mean:.3f}±{acc_sd:.3f}')

f1_arr = np.array(f1_list)
f1_mean= np.mean(f1_arr)
f1_sd = np.std(f1_arr)
print(f'f1: {f1_mean:.3f}±{f1_sd:.3f}')

roc_auc_arr = np.array(roc_auc_list)
roc_auc_mean= np.mean(roc_auc_arr)
roc_auc_sd = np.std(roc_auc_arr)
print(f'roc_auc: {roc_auc_mean:.3f}±{roc_auc_sd:.3f}')

print("Training Completed!")

Rep no 0, Fold no 0
Epoch: 1/300, train loss : 1.443204939365387, validation loss : 0.7870894074440002
Saving model...
Early stop counter: 0
Epoch: 2/300, train loss : 0.7968766838312149, validation loss : 0.722566545009613
Saving model...
Early stop counter: 0
Epoch: 3/300, train loss : 0.8732360899448395, validation loss : 0.681351900100708
Saving model...
Early stop counter: 0
Epoch: 4/300, train loss : 0.7404772341251373, validation loss : 0.6551194787025452
Saving model...
Early stop counter: 0
Epoch: 5/300, train loss : 0.6933062821626663, validation loss : 0.6939056515693665
Early stop counter: 1
Epoch: 6/300, train loss : 0.7090654969215393, validation loss : 0.6837751865386963
Early stop counter: 2
Epoch: 7/300, train loss : 0.684652104973793, validation loss : 0.6344522833824158
Saving model...
Early stop counter: 0
Epoch: 8/300, train loss : 0.6570419073104858, validation loss : 0.6143960356712341
Saving model...
Early stop counter: 0
Epoch: 9/300, train loss : 0.65999317169

# Next, we evaluated other methods for trf learning --> fine tuning the model with learning rate of 5x,10x and original learning rate
1. Using the best performance model --> pretrained at 60 epochs using mid similarity data, large dataset

## Fine tuning at 5x slower learning rate

In [5]:
train_data_root_path = './data/graph_data/data_oral_avail_train/'
train_data_raw_filename = 'data_oral_avail_train_50.csv'
test_data_root_path = './data/graph_data/data_oral_avail_test'
test_data_raw_filename = 'data_oral_avail_test_1_50.csv'
n_repetitions = 5
method_tf = 'fine_tune_5x'
params = best_params_vertical
es_trigger = 0
path_to_pretrained_model = './trf_learning_models/pretrained_models/vertical/mid_10x/'
path_to_save_trained_model = './trf_learning_models/trained_models/vertical/mid_10x/pretrained_40/'
path_to_trained_model = './trf_learning_models/trained_models/vertical/mid_10x/pretrained_40/'


bce_list = []
acc_list = []
f1_list = []
roc_auc_list = []

dataset_for_cv = LoadHOBDataset(train_data_root_path, train_data_raw_filename)
kf = KFold(n_splits=N_SPLITS)

for repeat in range(n_repetitions):
    for fold_no, (train_idx, valid_idx) in enumerate(kf.split(dataset_for_cv)):
        seed_everything(SEED_NO)
        train_dataset = []
        valid_dataset = []
        
        for t_idx in train_idx:
            train_dataset.append(
                torch.load(
                    f"./data/graph_data/data_oral_avail_train/processed/molecule_{t_idx}.pt"
                )
            )
        for v_idx in valid_idx:
            valid_dataset.append(
                torch.load(
                    f"./data/graph_data/data_oral_avail_train/processed/molecule_{v_idx}.pt"
                )
            )

        train_loader = DataLoader(
            train_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=True
        )
        valid_loader = DataLoader(
            valid_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=False
        )
        test_dataset = LoadHOBDataset(test_data_root_path, test_data_raw_filename)
        test_loader = DataLoader(
            test_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=False
        )
        print(f'Rep no {repeat}, Fold no {fold_no}')
        run_training(method_tf, train_loader, valid_loader, params, es_trigger, os.path.join(path_to_pretrained_model, f'pretrained_vertical_model_40_epoch.pt'),
            os.path.join(
                path_to_save_trained_model, f"trained_vertical_model_{method_tf}_repeat_{repeat}_fold_{fold_no}_{es_trigger}_es_trigger.pt"
            ),
        )

        bce, acc, f1, roc_auc = run_testing(method_tf, test_loader, params, 
                        os.path.join(path_to_save_trained_model, f"trained_vertical_model_{method_tf}_repeat_{repeat}_fold_{fold_no}_{es_trigger}_es_trigger.pt"))

        bce_list.append(bce)
        acc_list.append(acc)
        f1_list.append(f1)
        roc_auc_list.append(roc_auc)

bce_arr = np.array(bce_list)
mean_bce = np.mean(bce_arr)
sd_bce = np.std(bce_arr)
print(f'bce:{mean_bce:.3f}±{sd_bce:.3f}')

acc_arr = np.array(acc_list)
acc_mean= np.mean(acc_arr)
acc_sd = np.std(acc_arr)
print(f'acc:{acc_mean:.3f}±{acc_sd:.3f}')

f1_arr = np.array(f1_list)
f1_mean= np.mean(f1_arr)
f1_sd = np.std(f1_arr)
print(f'f1: {f1_mean:.3f}±{f1_sd:.3f}')

roc_auc_arr = np.array(roc_auc_list)
roc_auc_mean= np.mean(roc_auc_arr)
roc_auc_sd = np.std(roc_auc_arr)
print(f'roc_auc: {roc_auc_mean:.3f}±{roc_auc_sd:.3f}')

print("Training Completed!")

Rep no 0, Fold no 0
Epoch: 1/300, train loss : 1.344895452260971, validation loss : 0.727994441986084
Saving model...
Early stop counter: 0
Epoch: 2/300, train loss : 0.7425291836261749, validation loss : 0.6841184496879578
Saving model...
Early stop counter: 0
Epoch: 3/300, train loss : 0.7252871841192245, validation loss : 0.6768300533294678
Saving model...
Early stop counter: 0
Epoch: 4/300, train loss : 0.679559588432312, validation loss : 0.7253439426422119
Early stop counter: 1
Epoch: 5/300, train loss : 0.6757571548223495, validation loss : 0.6728647351264954
Saving model...
Early stop counter: 0
Epoch: 6/300, train loss : 0.6489957571029663, validation loss : 0.6296818256378174
Saving model...
Early stop counter: 0
Epoch: 7/300, train loss : 0.65016308426857, validation loss : 0.6208557486534119
Saving model...
Early stop counter: 0
Epoch: 8/300, train loss : 0.6353448033332825, validation loss : 0.6285421252250671
Early stop counter: 1
Epoch: 9/300, train loss : 0.621017441153

## Next we evaluated fine tune with 10x slower learning rate method

In [6]:
train_data_root_path = './data/graph_data/data_oral_avail_train/'
train_data_raw_filename = 'data_oral_avail_train_50.csv'
test_data_root_path = './data/graph_data/data_oral_avail_test'
test_data_raw_filename = 'data_oral_avail_test_1_50.csv'
n_repetitions = 5
method_tf = 'fine_tune_10x'
params = best_params_vertical
es_trigger = 0
path_to_pretrained_model = './trf_learning_models/pretrained_models/vertical/mid_10x/'
path_to_save_trained_model = './trf_learning_models/trained_models/vertical/mid_10x/pretrained_40/'
path_to_trained_model = './trf_learning_models/trained_models/vertical/mid_10x/pretrained_40/'


bce_list = []
acc_list = []
f1_list = []
roc_auc_list = []

dataset_for_cv = LoadHOBDataset(train_data_root_path, train_data_raw_filename)
kf = KFold(n_splits=N_SPLITS)

for repeat in range(n_repetitions):
    for fold_no, (train_idx, valid_idx) in enumerate(kf.split(dataset_for_cv)):
        seed_everything(SEED_NO)
        train_dataset = []
        valid_dataset = []
        
        for t_idx in train_idx:
            train_dataset.append(
                torch.load(
                    f"./data/graph_data/data_oral_avail_train/processed/molecule_{t_idx}.pt"
                )
            )
        for v_idx in valid_idx:
            valid_dataset.append(
                torch.load(
                    f"./data/graph_data/data_oral_avail_train/processed/molecule_{v_idx}.pt"
                )
            )

        train_loader = DataLoader(
            train_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=True
        )
        valid_loader = DataLoader(
            valid_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=False
        )
        test_dataset = LoadHOBDataset(test_data_root_path, test_data_raw_filename)
        test_loader = DataLoader(
            test_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=False
        )
        print(f'Rep no {repeat}, Fold no {fold_no}')
        run_training(method_tf, train_loader, valid_loader, params, es_trigger, os.path.join(path_to_pretrained_model, f'pretrained_vertical_model_40_epoch.pt'),
            os.path.join(
                path_to_save_trained_model, f"trained_vertical_model_{method_tf}_repeat_{repeat}_fold_{fold_no}_{es_trigger}_es_trigger.pt"
            ),
        )

        bce, acc, f1, roc_auc = run_testing(method_tf, test_loader, params, 
                        os.path.join(path_to_save_trained_model, f"trained_vertical_model_{method_tf}_repeat_{repeat}_fold_{fold_no}_{es_trigger}_es_trigger.pt"))

        bce_list.append(bce)
        acc_list.append(acc)
        f1_list.append(f1)
        roc_auc_list.append(roc_auc)

bce_arr = np.array(bce_list)
mean_bce = np.mean(bce_arr)
sd_bce = np.std(bce_arr)
print(f'bce:{mean_bce:.3f}±{sd_bce:.3f}')

acc_arr = np.array(acc_list)
acc_mean= np.mean(acc_arr)
acc_sd = np.std(acc_arr)
print(f'acc:{acc_mean:.3f}±{acc_sd:.3f}')

f1_arr = np.array(f1_list)
f1_mean= np.mean(f1_arr)
f1_sd = np.std(f1_arr)
print(f'f1: {f1_mean:.3f}±{f1_sd:.3f}')

roc_auc_arr = np.array(roc_auc_list)
roc_auc_mean= np.mean(roc_auc_arr)
roc_auc_sd = np.std(roc_auc_arr)
print(f'roc_auc: {roc_auc_mean:.3f}±{roc_auc_sd:.3f}')

print("Training Completed!")

Rep no 0, Fold no 0
Epoch: 1/300, train loss : 1.4199154525995255, validation loss : 0.7607229351997375
Saving model...
Early stop counter: 0
Epoch: 2/300, train loss : 0.7495656907558441, validation loss : 0.716059148311615
Saving model...
Early stop counter: 0
Epoch: 3/300, train loss : 0.777456983923912, validation loss : 0.6613466143608093
Saving model...
Early stop counter: 0
Epoch: 4/300, train loss : 0.6815161556005478, validation loss : 0.6844390034675598
Early stop counter: 1
Epoch: 5/300, train loss : 0.6848688274621964, validation loss : 0.7111064791679382
Early stop counter: 2
Epoch: 6/300, train loss : 0.6758259385824203, validation loss : 0.6560301780700684
Saving model...
Early stop counter: 0
Epoch: 7/300, train loss : 0.6521590948104858, validation loss : 0.6214882135391235
Saving model...
Early stop counter: 0
Epoch: 8/300, train loss : 0.6427769958972931, validation loss : 0.6139304041862488
Saving model...
Early stop counter: 0
Epoch: 9/300, train loss : 0.632296457

### Fine tune with normal learning rate

In [10]:
train_data_root_path = './data/graph_data/data_oral_avail_train/'
train_data_raw_filename = 'data_oral_avail_train_50.csv'
test_data_root_path = './data/graph_data/data_oral_avail_test'
test_data_raw_filename = 'data_oral_avail_test_1_50.csv'
n_repetitions = 5
method_tf = 'fine_tune'
params = best_params_vertical
es_trigger = 0
path_to_pretrained_model = './trf_learning_models/pretrained_models/vertical/mid_10x/'
path_to_save_trained_model = './trf_learning_models/trained_models/vertical/mid_10x/pretrained_40/'
path_to_trained_model = './trf_learning_models/trained_models/vertical/mid_10x/pretrained_40/'


bce_list = []
acc_list = []
f1_list = []
roc_auc_list = []

dataset_for_cv = LoadHOBDataset(train_data_root_path, train_data_raw_filename)
kf = KFold(n_splits=N_SPLITS)

for repeat in range(n_repetitions):
    for fold_no, (train_idx, valid_idx) in enumerate(kf.split(dataset_for_cv)):
        seed_everything(SEED_NO)
        train_dataset = []
        valid_dataset = []
        
        for t_idx in train_idx:
            train_dataset.append(
                torch.load(
                    f"./data/graph_data/data_oral_avail_train/processed/molecule_{t_idx}.pt"
                )
            )
        for v_idx in valid_idx:
            valid_dataset.append(
                torch.load(
                    f"./data/graph_data/data_oral_avail_train/processed/molecule_{v_idx}.pt"
                )
            )

        train_loader = DataLoader(
            train_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=True
        )
        valid_loader = DataLoader(
            valid_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=False
        )
        test_dataset = LoadHOBDataset(test_data_root_path, test_data_raw_filename)
        test_loader = DataLoader(
            test_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=False
        )
        print(f'Rep no {repeat}, Fold no {fold_no}')
        run_training(method_tf, train_loader, valid_loader, params, es_trigger, os.path.join(path_to_pretrained_model, f'pretrained_vertical_model_40_epoch.pt'),
            os.path.join(
                path_to_save_trained_model, f"trained_vertical_model_{method_tf}_repeat_{repeat}_fold_{fold_no}_{es_trigger}_es_trigger.pt"
            ),
        )

        bce, acc, f1, roc_auc = run_testing(method_tf, test_loader, params, 
                        os.path.join(path_to_save_trained_model, f"trained_vertical_model_{method_tf}_repeat_{repeat}_fold_{fold_no}_{es_trigger}_es_trigger.pt"))

        bce_list.append(bce)
        acc_list.append(acc)
        f1_list.append(f1)
        roc_auc_list.append(roc_auc)

bce_arr = np.array(bce_list)
mean_bce = np.mean(bce_arr)
sd_bce = np.std(bce_arr)
print(f'bce:{mean_bce:.3f}±{sd_bce:.3f}')

acc_arr = np.array(acc_list)
acc_mean= np.mean(acc_arr)
acc_sd = np.std(acc_arr)
print(f'acc:{acc_mean:.3f}±{acc_sd:.3f}')

f1_arr = np.array(f1_list)
f1_mean= np.mean(f1_arr)
f1_sd = np.std(f1_arr)
print(f'f1: {f1_mean:.3f}±{f1_sd:.3f}')

roc_auc_arr = np.array(roc_auc_list)
roc_auc_mean= np.mean(roc_auc_arr)
roc_auc_sd = np.std(roc_auc_arr)
print(f'roc_auc: {roc_auc_mean:.3f}±{roc_auc_sd:.3f}')

print("Training Completed!")

Rep no 0, Fold no 0
Epoch: 1/300, train loss : 1.1120635271072388, validation loss : 0.6694718599319458
Saving model...
Early stop counter: 0
Epoch: 2/300, train loss : 0.6994268596172333, validation loss : 0.683775782585144
Early stop counter: 1
Epoch: 3/300, train loss : 0.6763473153114319, validation loss : 0.6425277590751648
Saving model...
Early stop counter: 0
Epoch: 4/300, train loss : 0.6448647081851959, validation loss : 0.6277408003807068
Saving model...
Early stop counter: 0
Epoch: 5/300, train loss : 0.6334510743618011, validation loss : 0.5868958830833435
Saving model...
Early stop counter: 0
Epoch: 6/300, train loss : 0.6218878924846649, validation loss : 0.5972048044204712
Early stop counter: 1
Epoch: 7/300, train loss : 0.6114553958177567, validation loss : 0.5984869599342346
Early stop counter: 2
Epoch: 8/300, train loss : 0.6026855260133743, validation loss : 0.5806021690368652
Saving model...
Early stop counter: 0
Epoch: 9/300, train loss : 0.5857964605093002, valida